# Blatt 4

Hier ist das [Blatt](DBS_SS23_Uebungsblatt-04.pdf). Fangen wir mit der Definition von einem Full-Outer-Join an (brought to you by ChatGPT).

## Aufgabe 4

#### 1. Was ist ein Full Outer Join?

Ein Full Outer Join ist eine Art von Join-Operation in einer Datenbank, die Datensätze aus zwei Tabellen basierend auf einer gemeinsamen Spalte oder Bedingung kombiniert. Im Gegensatz zu anderen Joins wie dem Inner Join oder dem Left Outer Join, gibt der Full Outer Join alle Zeilen aus beiden Tabellen zurück, auch wenn es keine Übereinstimmungen zwischen den Tabellen gibt.

Die Funktionsweise eines Full Outer Joins lässt sich am besten anhand eines Beispiels erklären. Angenommen, wir haben zwei Tabellen: "Tabelle A" und "Tabelle B". Beide Tabellen enthalten eine Spalte namens "ID", die als gemeinsame Spalte fungiert.

Der Full Outer Join kombiniert die Datensätze beider Tabellen basierend auf der ID-Spalte. Dabei gibt es drei mögliche Szenarien:

1. Wenn eine ID in beiden Tabellen vorhanden ist, werden die entsprechenden Zeilen aus beiden Tabellen kombiniert.
2. Wenn eine ID nur in Tabelle A vorhanden ist, wird die Zeile aus Tabelle A beibehalten und die Werte in den Spalten von Tabelle B werden mit NULL-Werten aufgefüllt.
3. Wenn eine ID nur in Tabelle B vorhanden ist, wird die Zeile aus Tabelle B beibehalten und die Werte in den Spalten von Tabelle A werden mit NULL-Werten aufgefüllt.

Das Ergebnis des Full Outer Joins ist eine kombinierte Tabelle mit allen Zeilen aus beiden Tabellen, wobei NULL-Werte verwendet werden, wenn keine Übereinstimmung in der gemeinsamen Spalte gefunden wird.

#### 2. Wie kann man einen Full Outer Join in MariaDB simulieren?

Man kann einen Right-Join und einen Left-Join machen. Danach verbindet man beide per Union.

```sql
use uniDB;
select * from Assistant left join Professor P on P.PerNo = Assistant.Boss
union
select * from Assistant right join Professor P2 on P2.PerNo = Assistant.Boss;
```

#### 3. FOJ für Student und Kurs:

Man macht das wie folgt: Man holt sich alle Spalten aus dem linken Join der Student-Tabelle mit der attend-Tabelle, dadurch bekommen wir eine Tabelle wo auf jeden Fall jeder Student vertreten ist, aber nicht jeder Student ist in attend vertreten. Mit dieser Tabelle machen wir einen linken Join zu Course, um uns die Informationen zu den Kursen zu holen. Auf der anderen Seite machen wir einen rechten Join von Student auf attend, das gibt uns alle Spalten aus attend, womöglich nicht alle aus Student. Dann machen wir einen rechten Join auf Course, d.h. es sind mindestens alle Kurse in der Tabelle vertreten, aber nicht mehr alle Studenten.
```sql
select * from Student left join attend a on Student.StuNo = a.StuNo left join Course C on a.CouNo = C.CouNo
union
select * from Student right join attend a on Student.StuNo = a.StuNo right join Course C on a.CouNo = C.CouNo;
```

Erstellen der Room-Tabelle, wir erstellen für die beiden ForeignKeys die zugehörigen Spalten; durch den Check wird sichergegangen, dass mindestens eine der Spalte bei einem neuen Tupel leer ist (XOR). Danach fügen wir der neu erstellten Tabelle die Tupel aus Professor und löschen die Spalte Room aus Professor:
```mariadb
create table Room
(
    Build_No int default 1,
    Room_No int not null,
    Per_No_Prof int null,
    Per_No_Assi int null,
    foreign key (Per_No_Prof) references Professor(PerNo) on delete set null,
    foreign key (Per_No_Assi) references Assistant(PerNo) on delete set null,
    check ( (Per_No_Assi is not null and Per_No_Prof is null) or (Room.Per_No_Prof is not null and Room.Per_No_Assi is null))
);

insert into Room (Per_No_Prof, Room_No) (select PerNo, Room from Professor);
alter table Professor drop column Room;
commit;
```

insert into teach (TaughtBy, CouNo) (SELECT TaugthBy, CouNo From Professor RIGHT JOIN Course C on Professor.PerNo = C.TaugthBy)